In [23]:
import yaml
import os
# !pip install e3nn
from e3nn import o3

In [24]:
def load_config(config_file_path):
    with open(config_file_path, 'r') as stream:
        try:
            config = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return config

def save_config(config, config_file_path):
    with open(config_file_path, 'w') as stream:
        try:
            yaml.dump(config, stream, default_flow_style=False)
        except yaml.YAMLError as exc:
            print(exc)

def update_config(config, **kwargs):
    for key, value in kwargs.items():
        config[key] = value
    return config

def irreps(lvalue,num_features=32,even=False):
    return(str(o3.Irreps(
                    [
                        (num_features, (l, p))
                        for p in ((1, -1) if not even else (1,))
                        for l in range(lvalue + 1)
                    ]
                )))

In [26]:
# example_config_file_path="old_configs/aspirin.yaml"
# num_features=32
# max_epochs=200
# root="results/aspirin3"
# out_dir_configs="aspirin_configs"

# if not os.path.exists(out_dir_configs):
#     os.makedirs(out_dir_configs)

# !rm -rf {out_dir_configs}
# !mkdir {out_dir_configs}

# # n_val=50
# # for n_train in [900]:
# #     for conv0_layers in [2,3]:
# #         for lmax in [1,2,3]:

# #             config = load_config(example_config_file_path)

# #             num_layers=4-conv0_layers
# #             feature_irreps_hidden=str(o3.Irreps(
# #                             [
# #                                 (num_features, (l, p))
# #                                 for p in ((1, -1) if True else (1,))
# #                                 for l in range(lmax + 1)
# #                             ]
# #                         ))
# #             run_name=f"ntrain_{n_train}_lmax_{lmax}_invariant_{conv0_layers}_epochs_{max_epochs}"

# #             config = update_config(config, n_val=n_val,num_layers=num_layers, feature_irreps_hidden=feature_irreps_hidden, max_epochs=max_epochs, root=root, run_name=run_name, conv0_layers=conv0_layers, n_train=n_train)
# #             save_config(config, f"{out_dir_configs}/{run_name}.yaml")

In [32]:
example_config_file_path="old_configs/aspirin.yaml"
num_features=32
max_epochs=200
root="results/aspirin3"
out_dir_configs="aspirin_configs"
n_val=50
l_values=[1,0,0,0]

num_layers=len(l_values)
layer_irreps=",".join([irreps(lvalue,num_features,even=(lvalue==0)) for lvalue in l_values])
feature_irreps_hidden=""

if not os.path.exists(out_dir_configs):
    os.makedirs(out_dir_configs)

# !rm -rf {out_dir_configs}
# !mkdir {out_dir_configs}

for n_train in [900]:

    config = load_config(example_config_file_path)

    run_name=f"ntrain_{n_train}_irreps_{''.join([str(l) for l in l_values])}_epochs_{max_epochs}"

    config = update_config(config, n_val=n_val,num_layers=num_layers, max_epochs=max_epochs, root=root, run_name=run_name,layer_irreps=layer_irreps, n_train=n_train, feature_irreps_hidden=feature_irreps_hidden)
    save_config(config, f"{out_dir_configs}/{run_name}.yaml")